## GTP API를 사용하여 Fine-tuning 시켜보자
- fine-tuning에 필요한 데이터셋은 ChatCompletions API와 동일한 형식이 '대화'여야 하며, 각  메세지에 role별로 content가 들어간 메세지 목록이어야 함(system, user, assistants)
- fine-tuning시 최소 10개 이상의 대화 데이터가 필요하며, 비용상 처음부터 많은 양으로 학습시키기 보다는 50개 미만의 소량의 데이터 셋으로 학습시켜 결과를 확인해보고 필요시 추가하는 전략이 필요함
- openai 공식 페이지에서는 **100개 미만의 데이터**를 `소량의 데이터`라고 정의하고 있음
- 또한, 적어도 일부 학습 데이터에는 **기존 모델이 원하는대로 작동하지 않았을 경우**에 `제대로된 응답이 들어간 데이터`가 fine-tuning에 포함되어야 성능 향상을 크게 확인할 수 있음
    - 원하는대로 동작하지 않았던 응답

```
      Q : 프랑스의 수도는 어디야?
      A : 프랑스는 세계에서 가장 유명한 관광도시예요!
```
    
    - fine-tuning에 넣어줄 제대로된 데이터

```
      Q : 프랑스의 수도는 어디야?
      A : 프랑스의 수도는 파리입니다.
```


In [19]:
!pip install tiktoken

In [23]:
from openai import OpenAI
from getpass import getpass
from datetime import datetime
from collections import defaultdict
import numpy as np
import time
import json
import tiktoken # openai 모델들이 사용하는 base 토큰화 + 인코딩 모듈

In [27]:
MY_API_KEY = getpass.getpass("OpenAI API Key :")

OpenAI API Key : ········


In [31]:
client = OpenAI(api_key=MY_API_KEY)

### 1.Fine-tuning 전 데이터 로드 및 검증
- fine-tuning은 비용이 많이들 수 있기 때문에 사전에 적합한 데이터 형태인지 검증하고 진행하는 것이 좋음
- OpenAI fine-tuning API 사용시에 데이터의 구조는 JSONL 형태여야함
- **JSONL(JSON Lines)** : 한 줄당 json 객체가 존재하는 형태로 일반 json에 비해 한 줄씩 또는 청크 단위로 읽고 쓰는 것이 더 효율적이며 대규모의 데이터세트에 적합함

In [38]:
data = [
    {"messages" : [
        {"role" : "system", "content" : "You are a chatbot that gives clear answers."},
        {"role" : "user", "content" : "What can i do here?"},
        {"role" : "assistant", "content" : "Ask a question about he part you want."}
    ]},
    {"messages" : [
        {"role" : "system", "content" : "Your are a chatbot that gives clear answers"},
        {"role" : "user", "content" : "I fell sick"},
        {"role" : "assistant", "content" : "Please describe your symptoms in detail"}
    ]}
]
with open("finetune_data/my_dataset.jsonl", "w", encoding="utf-8") as f :
    # JSONL 파일 저장시 한 줄에 JSON 객체(딕셔너리)를 하나씩 저장해주는 방식
    for i in data :
        f.write(json.dumps(1) + "\n")

FileNotFoundError: [Errno 2] No such file or directory: 'finetune_data/my_dataset.jsonl'